# Migration between Scotland and overseas by age

Tabs: `SYOA Persons (2001-)` and `SYOA Males (2001-)`

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.nrscotland.gov.uk/statistics-and-data/statistics/' \
                      'statistics-by-theme/migration/migration-statistics/migration-between-scotland-and-overseas')
#     scraper.run()
    distribution = scraper.distribution(
        mediaType='application/vnd.ms-excel',
        title='Migration between Scotland and overseas by age')
    display(distribution)
    tabs = distribution.as_databaker()

In [2]:
Final_table = pd.DataFrame()

In [3]:
tab = [t for t in tabs if t.name == 'SYOA Persons (2001-)'][0]
observations = tab.excel_ref("B6").expand(RIGHT).expand(DOWN).is_not_blank().is_not_whitespace()
age = tab.excel_ref('C5').expand(RIGHT).is_not_blank()
flow = tab.excel_ref('A').expand(DOWN).by_index([3,23,43])
year = tab.excel_ref('A6').expand(DOWN).filter(contains_string('-')).is_not_blank().is_not_whitespace() - tab.excel_ref('A63').expand(DOWN)
Dimensions = [
            HDim(year,'Mid Year',DIRECTLY, LEFT),
            HDim(flow,'Flow',CLOSEST, ABOVE),
            HDim(age,'Age',DIRECTLY, ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People'),
            HDimConst('Sex','T'),
            HDimConst('Domestic geography','Scotland')
    ]
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
new_table = c1.topandas()
new_table['Value'] = pd.to_numeric(new_table['OBS'], errors='coerce').fillna(0)
new_table['Value'] = new_table['Value'].astype(int)
new_table['Flow'] = new_table['Flow'].map(
    lambda x: {
        'In migration of persons from overseas 2001-02 to latest' : 'Inflow', 
        'Out migration of persons to overseas 2001-02 to latest' : 'Outflow',
        'Net migration of persons from overseas 2001-02 to latest': 'Balance' 
        }.get(x, x))
new_table['Age'] = new_table['Age'].astype(str)
new_table['Age'] = new_table['Age'].map(lambda cell:cell.replace('None', 'all'))
new_table['Age'] = 'year/' + new_table['Age']
new_table['Age'] = new_table['Age'].map(lambda cell:cell.replace('.0', ''))
new_table.dropna(subset=['Mid Year'], inplace=True)
new_table['Mid Year'] = new_table['Mid Year'].map(lambda x: str(x)[0:4]) + '-06-30T00:00:00/P1Y'
Final_table = pd.concat([Final_table, new_table])

In [4]:
Final_table['Foreign geography'] = 'nrs/overseas'

In [5]:
Final_table = Final_table[['Domestic geography','Foreign geography','Mid Year','Sex','Age','Flow','Measure Type','Value','Unit']]